In [1]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
data_file = './/Amazon_Unlocked_Mobile.csv'
n = 413000  
s = 20000 
skip = sorted(random.sample(range(1,n),n-s))


df = pd.read_csv( data_file, delimiter = ",", skiprows = skip)

In [3]:
df

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,It's battery life is great. It's very responsi...,0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,"as described, fast ship!",0.0
3,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,5,Nokia Asha 302 Unlocked GSM Phone with 3.2MP C...,13.0
4,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,1,The battery died in less than 24/hours....,0.0
...,...,...,...,...,...,...
20835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
20836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
20837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
20838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


In [4]:
df.isnull().sum()

Product Name       0
Brand Name      3135
Price            301
Rating             0
Reviews            5
Review Votes     576
dtype: int64

In [5]:
df['Reviews']

0        It works good but it goes slow sometimes but i...
1        It's battery life is great. It's very responsi...
2                                 as described, fast ship!
3        Nokia Asha 302 Unlocked GSM Phone with 3.2MP C...
4               The battery died in less than 24/hours....
                               ...                        
20835                       another great deal great price
20836                                                   Ok
20837          Passes every drop test onto porcelain tile!
20838    I returned it because it did not meet my needs...
20839    Only downside is that apparently Verizon no lo...
Name: Reviews, Length: 20840, dtype: object

In [6]:
df = df.dropna(subset= ['Reviews'])

In [7]:
df

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,It's battery life is great. It's very responsi...,0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,"as described, fast ship!",0.0
3,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,5,Nokia Asha 302 Unlocked GSM Phone with 3.2MP C...,13.0
4,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,1,The battery died in less than 24/hours....,0.0
...,...,...,...,...,...,...
20835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
20836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
20837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
20838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


### Rating Mapping

In [8]:
Review_mapping =  {0: '0',1 : '0', 2 :'0',3 : '0', 4 : '1', 5 : '1'}
Review_mapping

{0: '0', 1: '0', 2: '0', 3: '0', 4: '1', 5: '1'}

In [9]:
Rating = lambda x: Review_mapping.get(x,x)
df['Rating']=df.Rating.map(Rating)

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
df['Rating'].unique()

array(['1', '0'], dtype=object)

### Text Analysis

In [11]:
df.columns

Index(['Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews',
       'Review Votes'],
      dtype='object')

In [12]:
a = df.iloc[:,3]
print(a)

0        1
1        0
2        1
3        1
4        0
        ..
20835    1
20836    0
20837    1
20838    0
20839    1
Name: Rating, Length: 20835, dtype: object


In [13]:
b = df.iloc[:,4]

In [14]:
doc = pd.concat([a,b], axis = 1)

In [15]:
train_docs, test_docs = train_test_split(doc, test_size = 0.3)

In [16]:
def review_to_wordlist(review, remove_stopwords = True):

    
    review_text = re.sub('[^a-zA-Z]'," ", review)
    #review_text에 영어만 넣기 
    
    words = review_text.lower().split()
    #소문자로 바꿔주고 그것들을 분리해준다 
    
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        #stops에 영어의 불용어를 넣어줌 
        #words는 소문자로 변환되고 띄어져있는 것이며 
        #stops에 있는 불용어를 제외하고 넣어줌 
        
    b = []
    stemmer = english_stemmer
    for word in words:
        b.append(stemmer.stem(word))
        #words에 전처리된것들의 어간들만 추출 
        
    return(b)


In [17]:
clean_reviews = []
for review in doc['Reviews']:
    clean_reviews.append( " ".join(review_to_wordlist(review)))

In [18]:
c = clean_reviews
type(c)

list

In [19]:
c = pd.DataFrame(clean_reviews)
c.columns = ['Reviews']

In [20]:
df = pd.concat([c,doc['Rating']], axis = 1)

In [21]:
df = df.dropna()

In [22]:
df.isnull().sum()

Reviews    0
Rating     0
dtype: int64

# ================ 5/1

### Tf-Idf

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from nltk.corpus import words

vectorizer = CountVectorizer(analyzer = 'word', 
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3),
                             vocabulary = set(words.words()), # nltk의 words를 사용하거나 문서 자체의 사전을 만들거나 선택한다. 
                             max_features = 20000
                            )

pipeline = Pipeline([
    ('vect', vectorizer),
    ('tfidf', TfidfTransformer(smooth_idf = False))
])

X_train_tf_idf_vector = pipeline.fit_transform(X_train)

X_test_tf_idf_vector = pipeline.fit_transform(X_test)

### ====================== 사용 x

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
NGRAM_RANGE = (1,2)
TOP_K = 20000
TOKEN_MODE = 'word'
MIN_DOC_FREQ = 2

In [24]:
def ngram_vectorize(texts, labels):
    kwargs = {
        'ngram_range' : NGRAM_RANGE,
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : TOKEN_MODE,
        'min_df' : MIN_DOC_FREQ
    }
    
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    transformed_texts = tfidf_vectorizer.fit_transform(texts)
    
    selector = SelectKBest(f_classif, k = min(TOP_K, transformed_texts.shape[1]))
    selector.fit(transformed_texts, labels)
    
    transformed_texts = selector.transform(transformed_texts).astype('float32')
    return transformed_texts

In [25]:
vect_data = ngram_vectorize(df['Reviews'], df['Rating'])
vect_data

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


<20830x20000 sparse matrix of type '<class 'numpy.float32'>'
	with 260662 stored elements in Compressed Sparse Row format>

In [26]:
X = vect_data.toarray()
y = (np.array(df['Rating']))

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=13)

### Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=2018, verbose=0,
                       warm_start=False)

In [29]:
%time forest = forest.fit(X_train, y_train)

CPU times: user 26min 44s, sys: 4.65 s, total: 26min 48s
Wall time: 7min 31s


In [30]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [31]:
k_fold = KFold(n_splits = 5, shuffle = True, random_state = 2018)

score = np.mean(cross_val_score(\
                               forest, X_train, \
                               y_train, cv = k_fold, scoring = 'roc_auc', n_jobs = -1))

In [32]:
format(score)

'0.5613361289299188'

In [33]:
result = forest.predict(X_test)

In [34]:
y_test

array(['0', '0', '1', ..., '1', '0', '1'], dtype=object)

In [35]:
a = sum(y_test == result)
print("테스트 셋 정확도 :",a / len(y_test))

테스트 셋 정확도 : 0.6670667306769084


### XGBOOST

In [36]:
import xgboost as xgb 
from xgboost import plot_importance 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from xgboost import XGBClassifier

In [37]:
dtrain = xgb.DMatrix(data = X_train, label = y_train)
dtest = xgb.DMatrix(data= X_test, label = y_test)

#numpy형태의 학습 데이터셋과 테스트 데이터셋을 DMatirx로 변환

In [38]:
params = {'max_depth' : 3,
          'eta' : 0.1,
          'objective' : 'binary:logistic',
          'eval_metric' : 'logloss',
          'early_stoppings' : 100}

num_rounds = 400
#eta : 학습률 
#손실함수 평가지표 : logloss

In [39]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]
%time xgb_model = xgb.train(params = params, 
                      dtrain = dtrain, 
                      num_boost_round = num_rounds,
                      evals = wlist)
#돌아가는것을 확인 

[17:22:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { early_stoppings } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.67880	eval-logloss:0.67895
[1]	train-logloss:0.66713	eval-logloss:0.66722
[2]	train-logloss:0.65761	eval-logloss:0.65786
[3]	train-logloss:0.64980	eval-logloss:0.65019
[4]	train-logloss:0.64333	eval-logloss:0.64372
[5]	train-logloss:0.63798	eval-logloss:0.63861
[6]	train-logloss:0.63353	eval-logloss:0.63449
[7]	train-logloss:0.62983	eval-logloss:0.63089
[8]	train-logloss:0.62679	eval-logloss:0.62805
[9]	train-logloss:0.62422	eval-logloss:0.62550
[10]	train-logloss:0.62211	eval-logloss:0.62363
[11]	train-logloss:0.62031	eval-logloss:0.62213
[12]	train-logloss:0.61880	eval-logloss:0.62091
[13]	train-logloss:0.

[162]	train-logloss:0.58912	eval-logloss:0.61540
[163]	train-logloss:0.58900	eval-logloss:0.61540
[164]	train-logloss:0.58888	eval-logloss:0.61534
[165]	train-logloss:0.58879	eval-logloss:0.61534
[166]	train-logloss:0.58866	eval-logloss:0.61536
[167]	train-logloss:0.58857	eval-logloss:0.61535
[168]	train-logloss:0.58848	eval-logloss:0.61538
[169]	train-logloss:0.58835	eval-logloss:0.61534
[170]	train-logloss:0.58827	eval-logloss:0.61528
[171]	train-logloss:0.58815	eval-logloss:0.61522
[172]	train-logloss:0.58803	eval-logloss:0.61519
[173]	train-logloss:0.58794	eval-logloss:0.61517
[174]	train-logloss:0.58784	eval-logloss:0.61516
[175]	train-logloss:0.58775	eval-logloss:0.61524
[176]	train-logloss:0.58759	eval-logloss:0.61518
[177]	train-logloss:0.58748	eval-logloss:0.61520
[178]	train-logloss:0.58734	eval-logloss:0.61526
[179]	train-logloss:0.58725	eval-logloss:0.61523
[180]	train-logloss:0.58710	eval-logloss:0.61535
[181]	train-logloss:0.58697	eval-logloss:0.61541
[182]	train-logloss:

[330]	train-logloss:0.57200	eval-logloss:0.61693
[331]	train-logloss:0.57190	eval-logloss:0.61693
[332]	train-logloss:0.57184	eval-logloss:0.61691
[333]	train-logloss:0.57177	eval-logloss:0.61690
[334]	train-logloss:0.57169	eval-logloss:0.61694
[335]	train-logloss:0.57162	eval-logloss:0.61693
[336]	train-logloss:0.57154	eval-logloss:0.61699
[337]	train-logloss:0.57144	eval-logloss:0.61697
[338]	train-logloss:0.57137	eval-logloss:0.61689
[339]	train-logloss:0.57130	eval-logloss:0.61691
[340]	train-logloss:0.57123	eval-logloss:0.61694
[341]	train-logloss:0.57116	eval-logloss:0.61695
[342]	train-logloss:0.57110	eval-logloss:0.61699
[343]	train-logloss:0.57102	eval-logloss:0.61698
[344]	train-logloss:0.57093	eval-logloss:0.61703
[345]	train-logloss:0.57085	eval-logloss:0.61703
[346]	train-logloss:0.57073	eval-logloss:0.61702
[347]	train-logloss:0.57066	eval-logloss:0.61706
[348]	train-logloss:0.57060	eval-logloss:0.61702
[349]	train-logloss:0.57043	eval-logloss:0.61694
[350]	train-logloss:

In [40]:
pred_probs = xgb_model.predict(dtest)
#예측값을 확률로 출력 
print('predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10], 3))

predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨
[0.686 0.692 0.699 0.699 0.717 0.699 0.697 0.679 0.73  0.679]


In [41]:
preds = [1 if x > 0.8 else 0 for x in pred_probs]

In [49]:
#Confusion Matrix, Accuracy, precision, recall, F1, AUC

def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    F1 = f1_score(y_test, preds)
    AUC = roc_auc_score(y_test, preds)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

In [50]:
get_clf_eval(y_test, preds)

TypeError: '<' not supported between instances of 'int' and 'str'

### LSTM

In [1]:
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [3]:
embed_size = 128
max_features = 6000


model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         768000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 64)          41216     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 810,537
Trainable params: 810,537
Non-trainable params: 0
________________________________________________

In [46]:
batch_size = 100
epochs = 3
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)


Train on 13331 samples, validate on 3333 samples
Epoch 1/3


KeyboardInterrupt: 

In [ ]:
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)

a = sum(y_test == prediction)
print("테스트 셋 정확도 :",a / len(y_test))